# A/B Testing Final Project - Udacity

## Step1: Choosing Metrics

### 1. Invariant Metrics: number of cookies, number of clicks and click-through-probability.

### 2. Evaluation Metrics: gross conversion, retention and net conversion. (Note : the number of user-id is not invariant metrics but it cannot be measured, so not in the evaluation metrics as well)

## Step2. Calculating Standard Deviation for evaluation matrics

#### Given a sample size of 5000 cookies visiting the course overview page.

In [185]:
import pandas as pd
import numpy as np
import scipy.stats

pageviews = 5000

In [186]:
df_basevals = pd.read_csv('/Users/pan/Desktop/A:B Testing/data/Final Project Baseline Values.csv',
                          index_col=False,header = None, names = ['metric','baseline_val'])
df_basevals

,metric,baseline_val
0,Unique cookies to view course overview page pe...,40000.000000
1,"Unique cookies to click ""Start free trial"" per...",3200.000000
2,Enrollments per day:,660.000000
3,"Click-through-probability on ""Start free trial"":",0.080000
4,"Probability of enrolling, given click:",0.206250
5,"Probability of payment, given enroll:",0.530000
6,"Probability of payment, given click",0.109313


In [187]:
df_basevals.metric = ['No_cookies', 'No_clicks', 'No_enrollments', 'CTP',
                      'p_enrolling_click', 'p_payment_enroll', 'p_paymet_click']
df_basevals

,metric,baseline_val
0,No_cookies,40000.000000
1,No_clicks,3200.000000
2,No_enrollments,660.000000
3,CTP,0.080000
4,p_enrolling_click,0.206250
5,p_payment_enroll,0.530000
6,p_paymet_click,0.109313


In [188]:
No_cookies = 40000
No_clicks = 3200
No_enrollments = 660
CTP = 0.08
p_enrolling_click = 0.20625
p_payment_enroll = 0.53
p_paymet_click = 0.109313

### SD of Gross Conversion

In [189]:
sd_GrossCvs = round(np.sqrt(p_enrolling_click * (1 - p_enrolling_click)/(pageviews * No_clicks / No_cookies)), 4)
sd_GrossCvs

0.0202

### SD of Retention

In [190]:
sd_retention = round(np.sqrt(p_payment_enroll * (1 - p_payment_enroll) / (pageviews * No_enrollments / No_cookies)), 4)
sd_retention

0.0549

### SD of Net Conversion

In [191]:
sd_netCvs = round(np.sqrt(p_paymet_click * (1 - p_paymet_click) / (pageviews * No_clicks / No_cookies)), 4)
sd_netCvs

0.0156

## Step 3: Calcuating number of pageviews

### Will you using bonferroni correction in your analysis phase? 
### - Yes, since there is more than one evaluation metrics.

In [193]:
alpha = 0.05
beta = 0.2
alpha_bv = alpha / 3
alpha_bv

0.016666666666666666

### Using the online calcuator - http://www.evanmiller.org/ab-testing/sample-size.html

### Gross Convesion
Baseline Conversion: 20.625%<br>
Minimum Detectable Effect: 1%<br>

In [194]:
size_grossCvs = 33014

### Retention
Baseline Conversion: 53%<br>
Minimum Detectable Effect: 1%<br>

In [195]:
size_retention = 50013

### Net Convesion
Baseline Conversion: 0.1093125%<br>
Minimum Detectable Effect: 0.75%<br>

In [196]:
size_netCvs = 35016

In [197]:
NO_groups = 2 #control and experiment groups

In [198]:
pageV_grossCvs = No_cookies * size_grossCvs * NO_groups / No_clicks
pageV_retention = No_cookies * size_retention * NO_groups / No_enrollments
pagev_netCvs = No_cookies * size_netCvs * NO_groups / No_clicks

In [199]:
data_pagev = {'metric' :['Gross Convesion', 'Retention', 'Net Convesion'], 'pageviews' : [pageV_grossCvs, pageV_retention, pagev_netCvs]}
df_pageview = pd.DataFrame( data_pagev)
df_pageview

,metric,pageviews
0,Gross Convesion,8.253500e+05
1,Retention,6.062182e+06
2,Net Convesion,8.754000e+05


In [200]:
traffic_exposed = 0.6
df_pageview['duration'] = df_pageview['pageviews'] / (traffic_exposed * No_cookies)
df_pageview

,metric,pageviews,duration
0,Gross Convesion,8.253500e+05,34.389583
1,Retention,6.062182e+06,252.590909
2,Net Convesion,8.754000e+05,36.475000


### 252.6 days is too long, so I will only take gross conversion and net conversion as evaluation metrics and using alpha_bv = alpha/2 recalculate #pagevies and duration.

In [201]:
alpha_bv = alpha/2

### Number of pageview is 875,400 and take 36.5 days to run the experiment.

## Step 4: Sanity Checks

In [202]:
df_control = pd.read_csv('/Users/pan/Desktop/A:B Testing/data/Final Project Results - Control.csv')
df_Experiment = pd.read_csv('/Users/pan/Desktop/A:B Testing/data/Final Project Results - Experiment.csv')
df_sanityChk = pd.DataFrame({'invarant metrics' :['No_cookies', 'No_clicks', 'TRP']})
df_control.head()

,Date,Pageviews,Clicks,Enrollments,Payments
0,"Sat, Oct 11",7723,687,134.0,70.0
1,"Sun, Oct 12",9102,779,147.0,70.0
2,"Mon, Oct 13",10511,909,167.0,95.0
3,"Tue, Oct 14",9871,836,156.0,105.0
4,"Wed, Oct 15",10014,837,163.0,64.0


In [203]:
total_cookies_control = sum(df_control['Pageviews'])
total_clicks_control = sum(df_control['Clicks'])
total_TRP_control = total_clicks_control / total_cookies_control

In [204]:
total_cookies_Experiment = sum(df_Experiment['Pageviews'])
total_clicks_Experiment = sum(df_Experiment['Clicks'])
total_TRP_Experiment = total_clicks_Experiment / total_cookies_Experiment

In [205]:
df_sanityChk['control'] =[total_cookies_control, total_clicks_control, total_TRP_control]
df_sanityChk['Experiment'] =[total_cookies_Experiment, total_clicks_Experiment, total_TRP_Experiment]
df_sanityChk['p'] = [0.5, 0.5, total_TRP_control]

In [206]:
sd_cookies = np.sqrt(0.5* 0.5 / (total_cookies_control + total_cookies_Experiment)) 
sd_clicks = np.sqrt(0.5* 0.5 / (total_clicks_control + total_clicks_Experiment)) 
sd_CTP = np.sqrt(total_TRP_control * (1 - total_TRP_control) / total_cookies_control)
df_sanityChk['sd'] = [sd_cookies, sd_clicks, sd_CTP]

In [207]:
z_value = scipy.stats.norm.ppf(1 - alpha_bv / 2)

In [208]:
df_sanityChk['margin'] = df_sanityChk['sd'] * z_value
df_sanityChk['lower_bound'] = df_sanityChk['p'] - df_sanityChk['margin']
df_sanityChk['upper_bound'] = df_sanityChk['p'] + df_sanityChk['margin']
df_sanityChk['observed'] = [total_cookies_Experiment / (total_cookies_Experiment + total_cookies_control),
                            total_clicks_Experiment / (total_clicks_Experiment + total_clicks_control),
                            total_TRP_Experiment]
df_sanityChk["PASS"] = df_sanityChk.apply(lambda x: (x.observed > x.lower_bound) and (x.observed < x.upper_bound),axis=1)
df_sanityChk

,invarant metrics,control,Experiment,p,sd,margin,lower_bound,upper_bound,observed,PASS
0,No_cookies,345543.000000,344660.000000,0.500000,0.000602,0.001349,0.498651,0.501349,0.499360,True
1,No_clicks,28378.000000,28325.000000,0.500000,0.002100,0.004706,0.495294,0.504706,0.499533,True
2,TRP,0.082126,0.082182,0.082126,0.000467,0.001047,0.081079,0.083173,0.082182,True


## Step 5: Effect Size Tests

In [209]:
df_effectTest = pd.DataFrame({'metric': ['Gross Conversion', 'Net Conversion']})

In [210]:
effect_control = df_control.dropna()
effect_exp = df_Experiment.dropna()
list(effect_control)

['Date', 'Pageviews', 'Clicks', 'Enrollments', 'Payments']

In [211]:
clicks_control = sum(effect_control['Clicks'])
enrollments_control = sum(effect_control['Enrollments'])
payment_control = sum(effect_control['Payments'])
grossCvs_control = enrollments_control / clicks_control
netCvs_control = payment_control / clicks_control

In [212]:
clicks_exp = sum(effect_exp['Clicks'])
enrollments_exp = sum(effect_exp['Enrollments'])
payment_exp = sum(effect_exp['Payments'])
grossCvs_exp = enrollments_exp / clicks_exp
netCvs_exp = payment_exp / clicks_exp

In [213]:
p_gross = (enrollments_control + enrollments_exp)/(clicks_control + clicks_exp)
p_net = (payment_control + payment_exp)/(clicks_control + clicks_exp)

In [214]:
sd_gross = np.sqrt(grossCvs_control*(1 -grossCvs_control)/clicks_control + grossCvs_exp*(1-grossCvs_exp)/clicks_exp)
sd_net = np.sqrt(netCvs_control*(1 - netCvs_control)/clicks_control + netCvs_exp*(1 - netCvs_exp)/clicks_exp)

In [215]:
df_effectTest['p_pool'] = [p_gross, p_net]
df_effectTest['sd'] = [sd_gross, sd_net]
df_effectTest['margin'] = df_effectTest['sd'] * z_value
df_effectTest['different'] = [grossCvs_exp - grossCvs_control, netCvs_exp - netCvs_control]
df_effectTest['lower_bound'] = df_effectTest['different'] - df_effectTest['margin']
df_effectTest['upper_bound'] = df_effectTest['different'] + df_effectTest['margin']

#### A metric is statistically significant if the confidence interval does not include 0 (that is, you can be confident there was a change),<br>it is practically significant if the confidence interval does not include the practical significance boundary (that is, you can be confident there is a change that matters to the business.)

In [216]:
df_effectTest['statistically significant'] = df_effectTest.apply(lambda x: (x.lower_bound > 0) or (x.upper_bound < 0),axis=1)
df_effectTest['d_min_upper'] = [0.01, 0.0075]
df_effectTest['d_min_lower'] = [-0.01, -0.0075]
df_effectTest['practically significant'] = df_effectTest.apply(lambda x: (x.lower_bound > x.d_min_upper) or (x.upper_bound < x.d_min_lower), axis = 1)
df_effectTest

,metric,p_pool,sd,margin,different,lower_bound,upper_bound,statistically significant,d_min_upper,d_min_lower,practically significant
0,Gross Conversion,0.208607,0.004370,0.009795,-0.020555,-0.030350,-0.010760,True,0.0100,-0.0100,True
1,Net Conversion,0.115127,0.003434,0.007697,-0.004874,-0.012571,0.002823,False,0.0075,-0.0075,False


## Step 6: Sign Tests

In [219]:
df_signTest = pd.DataFrame(effect_control['Date'])
df_signTest['grossCvs_control'] = effect_control['Enrollments'] / effect_control['Clicks']
df_signTest['grossCvs_exp'] = effect_exp['Enrollments'] / effect_exp['Clicks']
df_signTest['netCvs_control'] = effect_control['Payments'] / effect_control['Clicks']
df_signTest['netCvs_exp'] = effect_exp['Payments'] / effect_exp['Clicks']
df_signTest.head()

,Date,grossCvs_control,grossCvs_exp,netCvs_control,netCvs_exp
0,"Sat, Oct 11",0.195051,0.153061,0.101892,0.049563
1,"Sun, Oct 12",0.188703,0.147771,0.089859,0.115924
2,"Mon, Oct 13",0.183718,0.164027,0.104510,0.089367
3,"Tue, Oct 14",0.186603,0.166868,0.125598,0.111245
4,"Wed, Oct 15",0.194743,0.168269,0.076464,0.112981


In [226]:
df_signTest['grossCvs_sign'] = df_signTest['grossCvs_control'] - df_signTest['grossCvs_exp']
No_sucesses_grossCvs = len(df_signTest[df_signTest.grossCvs_sign > 0])
No_sucesses_grossCvs

19

In [227]:
df_signTest['netCvs_sign'] = df_signTest['netCvs_control'] - df_signTest['netCvs_exp']
No_sucesses_netCvs = len(df_signTest[df_signTest.netCvs_sign > 0])
No_sucesses_netCvs

13

In [228]:
No_trials = len(df_signTest)
No_trials

23

### Using the online calculator to calculate the two-tail p value. https://www.graphpad.com/quickcalcs/binomial1/

In [241]:
df_signTest_result = pd.DataFrame({'metric':['grossCvs', 'netCvs']})
df_signTest_result['p_value'] = [0.0026, 0.6776]

df_signTest_result['statistical significance'] = df_signTest_result['p_value'] < 0.5
df_signTest_result

,metric,p_value,statistical significance
0,grossCvs,0.0026,True
1,netCvs,0.6776,False
